In [ ]:
!pip install -q lime shap captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch.nn.functional as F
from lime import lime_text
import shap
from captum.attr import IntegratedGradients as CaptumIG
import concurrent.futures
import torch
import scipy as sp
import numpy as np
import time
import gc

In [ ]:
# df = pd.read_csv('Restaurant_reviews_preds.csv')
df = pd.read_csv('Hotel_reviews_preds.csv')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
def predictor(texts):
    local_tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = local_tokenizer(texts, return_tensors="pt", padding=True).to(device)
    outputs = model(**inputs)
    probas = F.softmax(outputs.logits, dim=1).detach().cpu().numpy()
    return probas

def tokenize(text):
    local_tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = local_tokenizer.tokenize(text)
    for i, token in enumerate(tokens):
        if token.startswith("##"):
            tokens[i] = tokens[i][2:]
    return tokens

In [ ]:
lime_explainer = lime_text.LimeTextExplainer(class_names=list(model.config.label2id.keys()), split_expression=tokenize, feature_selection='none')

In [ ]:
def lime_explain(text, label):
    label_id = model.config.label2id[label]
    local_tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = local_tokenizer.tokenize(text)
    reconstructed_text = local_tokenizer.decode(local_tokenizer.convert_tokens_to_ids(tokens))
    start = time.time()
    exp = lime_explainer.explain_instance(reconstructed_text, predictor, num_samples=50, labels=[label_id])
    end = time.time()
    lime_time = end - start
    lime_exp = dict(exp.as_list(model.config.label2id[label]))
    lime_exp = {token: lime_exp.get(token, lime_exp.get(token[2:], 0)) for token in tokens}
    return lime_exp, lime_time

In [ ]:
lime_exp, lime_time = lime_explain(df.iloc[0]["text"], df.iloc[0]["prediction"])
lime_exp, lime_time

({'المكان': -0.005646679102251189,
  'الذي': -0.031662424348775094,
  'يمكنك': -0.012045296343720672,
  'فيه': 0.005328327655523792,
  'مراجعة': -0.02599369048658645,
  'الذات': 0.013109012394794398,
  'والتف': 0.013367547731441786,
  '##كر': 0.03152645393122052,
  'هو': 0.003528780149967393,
  'كوك': -0.04074350063745887,
  '##روب': 0.0004994505891397601,
  '##يت': -0.006870991252231884,
  '،': 0.009269050611965162,
  'غانا': -0.007915520252512433,
  '.': 0.031922427747734024,
  'ثمة': -0.0036624058979693444,
  'الكثير': -0.01620331551442952,
  'عند': -0.011532017642066513,
  'زيارة': -0.011123740828230552,
  'وعلى': -0.014324653040228275,
  'الرغم': -0.012387911798253163,
  'من': -0.022289684486124128,
  'الفقر': -0.010612178387249541,
  'سوف': 0.0053911147904510525,
  'تلا': -0.01230867466194244,
  '##ح': -0.005933359899726308,
  '##ظه': -0.03512155973814297,
  'على': -0.0014627429550227483,
  'طريقك': -0.021305926906166794,
  'إلى': -0.01931574431135904,
  'بي': 0.01108128166442648

In [ ]:
polarities = list(model.config.id2label.values())
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device, top_k=None)
shap_explainer = shap.Explainer(pipe, tokenizer, algorithm="partition", output_names=polarities, max_evals=50)

Device set to use cuda


In [ ]:
def shap_explain(text, label):
    local_tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = local_tokenizer.tokenize(text)
    start = time.time()
    shap_values = shap_explainer([text])
    end = time.time()
    shap_time = end - start

    values = shap_values.values[0].tolist()[1:-1]
    polarity_idx = model.config.label2id[label]
    shap_exp = {}
    for token, value in zip(tokens, values):
        shap_exp[token] = value[polarity_idx]
    return shap_exp, shap_time

In [ ]:
shap_exp, shap_time = shap_explain(df.iloc[0]["text"], df.iloc[0]["prediction"])
shap_exp, shap_time

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
PartitionExplainer explainer: 2it [00:13, 13.91s/it]               


({'المكان': 0.0033228397737723524,
  'الذي': 0.0016835812673084275,
  'يمكنك': 0.00016811555863152872,
  'فيه': 0.0015643996993449298,
  'مراجعة': 0.0015643996993449296,
  'الذات': 0.0015643996993449296,
  'والتف': 0.0015643996993449296,
  '##كر': 0.0015643996993449296,
  'هو': 0.0015643996993449298,
  'كوك': 0.001652330785517212,
  '##روب': 0.001652330785517212,
  '##يت': 0.001652330785517212,
  '،': 0.0033228397737723524,
  'غانا': 0.0016835812673084275,
  '.': 0.004321016197399155,
  'ثمة': 0.0016835812673084277,
  'الكثير': 0.0016835812673084277,
  'عند': 0.001683581267308428,
  'زيارة': 0.001683581267308428,
  'وعلى': 0.0037781400306300186,
  'الرغم': 0.0016835812673084275,
  'من': 0.0037781400306300186,
  'الفقر': 0.0016835812673084275,
  'سوف': 0.0016835812673084275,
  'تلا': 0.0016835812673084275,
  '##ح': 0.0016835812673084275,
  '##ظه': 0.0016835812673084275,
  'على': 0.0033228397737723524,
  'طريقك': 0.0016835812673084275,
  'إلى': 0.0037781400306300186,
  'بي': 0.0034465897

In [ ]:
def forward_func(input_embeds):
    outputs = model(inputs_embeds=input_embeds)
    return torch.softmax(outputs.logits, dim=-1)

In [ ]:
ig = CaptumIG(forward_func)

In [ ]:
def ig_explain(text, label):
    local_tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = local_tokenizer.tokenize(text)
    input_ids = local_tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    with torch.no_grad():
        embeddings = model.get_input_embeddings()(input_ids)

    start = time.time()
    polarity_idx = model.config.label2id[label]
    attributions, _ = ig.attribute(
        inputs=embeddings,
        baselines=torch.zeros_like(embeddings).to(device),
        target=polarity_idx,  # Specify the class index
        return_convergence_delta=True,
    )
    attributions = attributions.sum(dim=-1).squeeze(0)
    scores = attributions.cpu().detach().numpy()
    ig_exp = {}
    for token, score in zip(tokens, scores):
        ig_exp[token] = score
    end = time.time()
    ig_time = end - start

    return ig_exp, ig_time

In [ ]:
ig_exp, ig_time = ig_explain(df.iloc[0]["text"], df.iloc[0]["prediction"])
ig_exp, ig_time

({'المكان': 0.0039353197651267715,
  'الذي': -0.0011355757700621271,
  'يمكنك': -0.008537085023784875,
  'فيه': -0.002654918349475747,
  'مراجعة': -0.03392617385945303,
  'الذات': 0.0061621843181141455,
  'والتف': -0.009756728119344357,
  '##كر': 0.001448176301547434,
  'هو': 0.0009425951556778251,
  'كوك': -0.01341310153376777,
  '##روب': -0.006286468373540022,
  '##يت': -0.006538712018345644,
  '،': -0.009980412573399919,
  'غانا': -0.019918631609139436,
  '.': -0.004265196240183321,
  'ثمة': 0.02026614062063495,
  'الكثير': 0.009800639555100524,
  'عند': 0.005994193082055902,
  'زيارة': 0.01623987049793878,
  'وعلى': 0.002890480108096442,
  'الرغم': -0.002085313902936637,
  'من': -0.0005679052357317,
  'الفقر': -0.02017030908535249,
  'سوف': 0.004973035550613085,
  'تلا': 0.0014307283518969868,
  '##ح': 0.00784366748408146,
  '##ظه': 0.004840868548802278,
  'على': 0.00843427001722901,
  'طريقك': 0.012813154652686657,
  'إلى': -0.000764267540117615,
  'بي': 0.0009075315160616688,
  '

In [ ]:
def exai_explain(text, label):
    start = time.time()
    # Use ThreadPoolExecutor to run the three methods concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        lime_future = executor.submit(lime_explain, text, label)
        shap_future = executor.submit(shap_explain, text, label)
        ig_future = executor.submit(ig_explain, text, label)

        lime_results, _ = lime_future.result()
        shap_results, _ = shap_future.result()
        ig_results, _ = ig_future.result()

    # Combine the results of the three methods by averaging the scores
    exai_exp = {}
    for token, (lime_score, shap_score, ig_score) in zip(lime_results.keys(), zip(lime_results.values(), shap_results.values(), ig_results.values())):
        exai_exp[token] = (lime_score + shap_score + ig_score) / 3

    end = time.time()
    exai_time = end - start

    return exai_exp, exai_time

In [ ]:
exai_exp, exai_time = exai_explain(df.iloc[0]["text"], df.iloc[0]["prediction"])
exai_exp, exai_time

({'المكان': 0.0012932399056518064,
  'الذي': 0.01600277398513504,
  'يمكنك': -0.006417805985935103,
  'فيه': 0.0013155910671541907,
  'مراجعة': -0.018150907896742494,
  'الذات': 0.009608498483350318,
  'والتف': 0.007683639666817141,
  '##كر': -0.006210083806889934,
  'هو': -0.004049526363978997,
  'كوك': 0.00042165783957841897,
  '##روب': -0.00011520774417664477,
  '##يت': -0.003709644193866503,
  '،': -0.00034235551519900924,
  'غانا': -0.0019653035964081415,
  '.': 0.014837484865897796,
  'ثمة': 0.007475202830465751,
  'الكثير': -0.0008080303163519095,
  'عند': 0.0008724783026379792,
  'زيارة': 0.008865085156460168,
  'وعلى': -0.0008447189622246214,
  'الرغم': 0.0018463052909307234,
  'من': 0.010742766648888305,
  'الفقر': -0.013291126998089852,
  'سوف': 0.005416477575035863,
  'تلا': -7.062026418491914e-06,
  '##ح': 0.0011745117275717015,
  '##ظه': 0.007788393434305846,
  'على': 0.015291746950210827,
  'طريقك': 0.0037530701030225873,
  'إلى': 0.0077498998644233525,
  'بي': 0.0021747

In [ ]:
# df_sample = pd.read_csv('Restaurant_reviews_xai.csv')
# df_sample = pd.read_csv('Hotel_reviews_xai.csv')

In [ ]:
# get 1000 sample rows where text is no longer than 1000 characters
df_sample = df[df["text"].apply(lambda x: len(x) < 1000)].sample(1000)

In [ ]:
len(df_sample)

2000

In [ ]:
df_sample["lime_exp"], df_sample["lime_time"] = zip(*df_sample.apply(lambda x: lime_explain(x["text"], x["prediction"]), axis=1))

In [ ]:
df_sample["shap_exp"], df_sample["shap_time"] = zip(*df_sample.apply(lambda x: shap_explain(x["text"], x["prediction"]), axis=1))

In [ ]:
df_sample["ig_exp"], df_sample["ig_time"] = zip(*df_sample.apply(lambda x: ig_explain(x["text"], x["prediction"]), axis=1))

In [ ]:
df_sample["exai_exp"], df_sample["exai_time"] = zip(*df_sample.apply(lambda x: exai_explain(x["text"], x["prediction"]), axis=1))

In [ ]:
df_sample.head()

In [ ]:
# df_sample.to_csv("Restaurant_reviews_xai.csv", index=False)
df_sample.to_csv("Hotel_reviews_xai.csv", index=False)